<a href="https://colab.research.google.com/github/wpcastro/cloudmart1/blob/main/Redeneural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação, Bibliotecas e Download do Banco de Dados

# Instala a biblioteca gdown para baixar arquivos do Google Drive
!pip install gdown -q

import pandas as pd
import gdown
import zipfile
import os

# --- Download e Descompactação do Dataset ---
# ID do arquivo público contendo o dataset Telco Customer Churn
id_arquivo = '1Me_iRxS_5v0IuGspS5g-C7wHn_vQVXWp'
output_zip = 'telco-churn.zip'

print("Baixando o banco de dados...")
gdown.download(id=id_arquivo, output=output_zip, quiet=False)

print("\nDescompactando o arquivo...")
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('./')

# --- Carregamento e Vistoria dos Dados ---
# O nome do arquivo dentro do zip é 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
caminho_csv = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
df_clientes = pd.read_csv(caminho_csv)

print("\nBanco de dados carregado com sucesso! As 5 primeiras linhas são:")
pd.set_option('display.max_columns', None) # Garante que todas as colunas sejam exibidas
print(df_clientes.head())

print(f"\nO banco de dados possui {df_clientes.shape[0]} linhas e {df_clientes.shape[1]} colunas.")

Instalação, Bibliotecas e Download do Banco de Dados

In [ ]:
# Pré-processamento e Preparação dos Dados

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Limpeza dos Dados ---
# A coluna 'TotalCharges' tem espaços em branco para clientes novos.
# Convertemos para numérico, forçando erros a se tornarem 'NaN' (Not a Number)
df_clientes['TotalCharges'] = pd.to_numeric(df_clientes['TotalCharges'], errors='coerce')

# Preenchemos os valores 'NaN' com a mediana da coluna, uma abordagem robusta.
df_clientes['TotalCharges'].fillna(df_clientes['TotalCharges'].median(), inplace=True)

# Removemos a ID do cliente, que não é útil para a previsão
df_clientes = df_clientes.drop('customerID', axis=1)

# --- Transformação de Dados ---
# Convertendo a coluna alvo 'Churn' de 'Yes'/'No' para 1/0
df_clientes['Churn'] = df_clientes['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convertendo todas as outras colunas de texto em colunas numéricas (One-Hot Encoding)
df_clientes_dummies = pd.get_dummies(df_clientes)

# --- Separação e Normalização ---
# Separando as features (X) do alvo (y)
X = df_clientes_dummies.drop('Churn', axis=1)
y = df_clientes_dummies['Churn']

# Normalizando as features para que a rede neural funcione melhor
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividindo os dados em 80% para treino e 20% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X_scaled, y, test_size=0.20, random_state=42, stratify=y)

print("Dados limpos, transformados e divididos com sucesso!")
print(f"Número de features após a transformação: {X.shape[1]}")

Pré-processamento e Preparação dos Dados

In [ ]:
# Treinar a Rede Neural para Prever Churn

from sklearn.neural_network import MLPClassifier

# Criando a rede neural. Uma arquitetura com duas camadas ocultas é um bom ponto de partida.
modelo_churn = MLPClassifier(hidden_layer_sizes=(16, 8), activation='relu', solver='adam', max_iter=500, random_state=42)

# Treinando o modelo com os dados de treino preparados
print("Iniciando o treinamento do modelo de previsão de churn...")
modelo_churn.fit(X_treino, y_treino)
print("Treinamento concluído!")

Treinar a Rede Neural para Prever Churn

In [ ]:
# Avaliação e Interpretação de Negócio

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# O modelo faz previsões no conjunto de teste (clientes que ele nunca viu)
previsoes = modelo_churn.predict(X_teste)

# 1. Acurácia Geral
acuracia = accuracy_score(y_teste, previsoes)
print(f"Acurácia Geral do Modelo: {acuracia:.2%}\n")

# 2. Relatório de Classificação Detalhado
# Observe o 'recall' para a classe 1: ele mostra a % de clientes com churn que o modelo conseguiu "capturar"
print("Relatório de Classificação:")
print(classification_report(y_teste, previsoes, target_names=['Ficou (0)', 'Saiu (1)']))

# 3. Matriz de Confusão com Interpretação de Negócio
print("--- Matriz de Confusão e Análise de Negócio ---")
matriz = confusion_matrix(y_teste, previsoes)

verdadeiros_negativos, falsos_positivos = matriz[0]
falsos_negativos, verdadeiros_positivos = matriz[1]

print(f"Clientes que ficaram e o modelo previu corretamente: {verdadeiros_negativos} (Acerto)")
print(f"Clientes que ficaram mas o modelo previu que sairiam: {falsos_positivos} (Erro - Custo de retenção desnecessário)")
print(f"Clientes que saíram mas o modelo previu que ficariam: {falsos_negativos} (Erro - Perda de receita inesperada)")
print(f"Clientes que saíram e o modelo previu corretamente: {verdadeiros_positivos} (Acerto - Oportunidade de ação!)")

print(f"\nInsight de Negócio: O modelo identificou corretamente {verdadeiros_positivos} clientes que iriam cancelar. A equipe de retenção pode agora focar seus esforços nesse grupo para tentar revertê-los.")

 Avaliação e Interpretação de Negócio